# Memory profiling - p122

In [1]:
import numpy as np
from imp import reload

%load_ext line_profiler

In [2]:
%%writefile _simulation.py

import numpy as np

def step(*shape):
    # Create random vector with +1 or -1
    return (2 * (np.random.random_sample(shape) < 0.5) - 1)

def simulate(iterations, n=10000):
    s = step(iterations, n)
    x = np.cumsum(s, axis=0)
    bins = np.arange(-30, 30, 1)
    y = np.vstack([np.histogram(x[i,:], bins)[0]
                  for i in range(iterations)])
    return y

Overwriting _simulation.py


In [3]:
import _simulation
reload(_simulation)

<module '_simulation' from '/media/sf_host/home2/yannick2/github/Spikes/Python/_simulation.py'>

In [4]:
%lprun -T lprof0 -f _simulation.simulate _simulation.simulate(500)


*** Profile printout saved to text file 'lprof0'. 


In [5]:
! cat lprof0

Timer unit: 1e-06 s

Total time: 0.421734 s
File: /media/sf_host/home2/yannick2/github/Spikes/Python/_simulation.py
Function: simulate at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           def simulate(iterations, n=10000):
     9         1        93818  93818.0     22.2      s = step(iterations, n)
    10         1       135031 135031.0     32.0      x = np.cumsum(s, axis=0)
    11         1           31     31.0      0.0      bins = np.arange(-30, 30, 1)
    12         1            4      4.0      0.0      y = np.vstack([np.histogram(x[i,:], bins)[0]
    13         1       192847 192847.0     45.7                    for i in range(iterations)])
    14         1            3      3.0      0.0      return y

In [6]:
%load_ext memory_profiler

import gc

gc.collect()

%mprun -T mprof0 -f _simulation.simulate _simulation.simulate(50, int(1e5))
! cat mprof0



*** Profile printout saved to text file mprof0. 
Filename: /media/sf_host/home2/yannick2/github/Spikes/Python/_simulation.py

Line #    Mem usage    Increment   Line Contents
     8     47.9 MiB      0.0 MiB   def simulate(iterations, n=10000):
     9     90.8 MiB     42.9 MiB       s = step(iterations, n)
    10    128.9 MiB     38.1 MiB       x = np.cumsum(s, axis=0)
    11    128.9 MiB      0.0 MiB       bins = np.arange(-30, 30, 1)
    12    128.9 MiB      0.0 MiB       y = np.vstack([np.histogram(x[i,:], bins)[0]
    13    128.9 MiB      0.0 MiB                     for i in range(iterations)])
    14    128.9 MiB      0.0 MiB       return y

In [7]:
%memit y = np.random.randn(1000,10000)

peak memory: 128.98 MiB, increment: 76.34 MiB


In [8]:
x = np.random.randn(10000, 1000)
%memit y = None
%time %memit x = x * 2
%time %memit x *= 2
%memit x[10,:] = x[11,:] * 2
%memit y = x[11,:] * 2
%memit y = y * 3
%memit x[:] = x[:] * 2

peak memory: 129.12 MiB, increment: -76.21 MiB
peak memory: 203.44 MiB, increment: 74.31 MiB
CPU times: user 49 ms, sys: 43 ms, total: 92 ms
Wall time: 227 ms
peak memory: 129.16 MiB, increment: 0.00 MiB
CPU times: user 30 ms, sys: 48 ms, total: 78 ms
Wall time: 212 ms
peak memory: 129.17 MiB, increment: 0.01 MiB
peak memory: 129.18 MiB, increment: 0.00 MiB
peak memory: 129.18 MiB, increment: 0.00 MiB
peak memory: 205.48 MiB, increment: 76.30 MiB


In [9]:
import scipy 
import scipy.sparse

N = 4000
s = 4000

def bigsparse (N, s):
    x = np.random.randint(0, N, s)
    y = np.random.randint(0, N, s)
    return scipy.sparse.csr_matrix((np.ones(s), (x,y)), (N,N))

sparse1 = None
sparse2 = None
%memit sparse1 = bigsparse(N, s)
foo = None
%time %memit foo = sparse1 * sparse1
%time print((foo != 0).sum())

%memit sparse2 = bigsparse(N, s * 100)
foo = None
%time %memit foo = sparse2 * sparse2
%time print((foo != 0).sum())

%memit nonsparse_m = sparse2.todense()
foo = None
%time %memit foo = nonsparse_m * nonsparse_m
%time print((foo != 0).sum())

peak memory: 130.45 MiB, increment: 0.12 MiB
peak memory: 130.46 MiB, increment: 0.02 MiB
CPU times: user 28 ms, sys: 36 ms, total: 64 ms
Wall time: 213 ms
4002
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 663 µs
peak memory: 150.48 MiB, increment: 20.00 MiB
peak memory: 317.63 MiB, increment: 167.15 MiB
CPU times: user 349 ms, sys: 25 ms, total: 374 ms
Wall time: 524 ms
14605680
CPU times: user 757 ms, sys: 24 ms, total: 781 ms
Wall time: 833 ms
peak memory: 561.79 MiB, increment: 244.15 MiB
peak memory: 409.76 MiB, increment: 137.20 MiB
CPU times: user 5.59 s, sys: 535 ms, total: 6.12 s
Wall time: 2.41 s
14605680
CPU times: user 31 ms, sys: 71 ms, total: 102 ms
Wall time: 27.5 ms
